<a href="https://colab.research.google.com/github/lazyboneOwO/PL-Repo/blob/main/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium google-search-results gradio

In [ ]:
import requests
import folium
from serpapi import GoogleSearch

In [ ]:
SERPAPI_KEY = "e3766589a7992ed872b6b85c4d000ca2eeb0ae26425bb26b455a4acc98a8dd1f"

In [ ]:
def get_station_coordinates(station_name):
    """
    使用 SerpApi 搜尋捷運站 → 取得座標
    """
    params = {
        "engine": "google_maps",
        "q": station_name,
        "api_key": SERPAPI_KEY,
        "num":1,
        "hl": "zh-tw"
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    place = results.get("place_results", {})
    if not place:
        return None

    lat = place["gps_coordinates"]["latitude"]
    lon = place["gps_coordinates"]["longitude"]

    return lat, lon

In [ ]:
def search_vegan_nearby(lat, lon, radius=800):
    """
    搜尋指定座標附近 radius 公尺內的素食店家
    """
    params = {
        "engine": "google_maps",
        "type": "search",
        "google_domain": "google.com",
        "num":3,
        "hl": "zh-tw",
        "q": "素食",
        "ll": f"@{lat},{lon},15z",
        "api_key": SERPAPI_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    local_results = results.get("local_results", [])
    vegan_shops = []

    for shop in local_results:
        name = shop.get("title", "")
        lat2 = shop.get("gps_coordinates", {}).get("latitude")
        lon2 = shop.get("gps_coordinates", {}).get("longitude")

        if name and lat2 and lon2:
            vegan_shops.append({
                "name": name,
                "lat": lat2,
                "lon": lon2
            })

    return vegan_shops

In [ ]:
def create_map(station_name, station_lat, station_lon, shops):
    """
    建立 Folium 地圖，並標示站體位置＋素食店家
    """
    m = folium.Map(location=[station_lat, station_lon], zoom_start=15)

    # 標示捷運站
    folium.Marker(
        [station_lat, station_lon],
        popup=f"{station_name}（捷運站）",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

    # 標示素食店家
    for shop in shops:
        folium.Marker(
            [shop["lat"], shop["lon"]],
            popup=shop["name"],
            icon=folium.Icon(color="green")
        ).add_to(m)

    return m

In [ ]:
def search_station_and_vegan(station_name):
    # 第1步：找捷運站座標
    coords = get_station_coordinates(station_name)
    if not coords:
        return "找不到此捷運站，請確認名稱", None

    station_lat, station_lon = coords
    #print(f"找到捷運站座標: {station_lat}, {station_lon}")

    # 第2步：搜尋附近素食店家
    shops = search_vegan_nearby(station_lat, station_lon)
    #print(f"總共找到 {len(shops)} 家素食店家")

    # 第3步：地圖呈現
    result_map = create_map(station_name, station_lat, station_lon, shops)
    return f"找到 {len(shops)} 家素食店家", result_map._repr_html_()

In [ ]:
'''
station = "東門站"
map_obj = search_station_and_vegan(station)
map_obj
'''

'\nstation = "東門站"\nmap_obj = search_station_and_vegan(station)\nmap_obj\n'

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# 捷運站附近素食餐廳查詢")
    gr.Markdown("輸入任意台北捷運站名稱，系統會自動搜尋附近素食店家並標記於地圖。")

    input_station = gr.Textbox(label="輸入捷運站名稱（例：西門站、台北車站、忠孝復興站）")

    btn = gr.Button("搜尋附近素食店")

    output_text = gr.Textbox(label="搜尋結果摘要")
    output_map = gr.HTML(label="附近素食店家地圖")

    btn.click(search_station_and_vegan, inputs=input_station, outputs=[output_text, output_map])

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e83b6b32d0c002bd3d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
